In [9]:
import pandas as pd
import numpy as np
import re
data=pd.read_csv('artifacts/Cleaned_Indian_Food_Dataset.csv',usecols=['TranslatedRecipeName','Cleaned-Ingredients','TranslatedInstructions','Ingredient-count'])

In [10]:
def clean_ingredients(ingredient_string):
    # Remove anything inside parentheses
    cleaned_string = re.sub(r'\s*\(.*?\)\s*', '', ingredient_string)
    # Convert the cleaned string to a list
    return [item.strip() for item in cleaned_string.split(',')]

# Apply the function to the 'ingredients' column and create a new column 'Ingredients cleaned'
data['Processed_Ingredients'] = data['Cleaned-Ingredients'].apply(clean_ingredients)

In [11]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.llms.ollama import Ollama
llm = Ollama(model='llama2')
output_parser=StrOutputParser(
)
prompt=ChatPromptTemplate([
        ('system',"""You are a chef. The user will give you a list of ingredients. You have to create an Indian Dish using those
        ingredients only.
        Use the following template to give your output:
         Ingredients Used:
            - Ingredient 1
            - Ingredient 2
            - other ingredients
         Instructions:
            - Instruction 1
            - Instruction 2
         Do not output anything else apart from Ingredients Used and Instructions.
         """),
        ("user","Ingredients: {Ingredients}")

    ])

chain=prompt|llm|output_parser

response=chain.invoke({'Ingredients':data['Processed_Ingredients'][0]})


In [16]:
response

'Ingredients Used:\n\n* Salt\n* Amchur\n* Karela\n* Red chilli powder\n* Gram flour\n* Onion\n* Cumin seeds\n* Coriander powder\n* Turmeric powder\n\nInstructions:\n\n* Heat sunflower oil in a pan over medium heat.\n* Add cumin seeds and let them sizzle for a few seconds.\n* Add finely chopped onion and sauté until it is translucent.\n* Add karela (bitter gourd) and sauté for 2-3 minutes, or until it starts to soften.\n* Add salt, red chilli powder, coriander powder, and turmeric powder, and mix well.\n* Add gram flour (besan) and mix well.\n* Add amchur (dried mango powder) and mix well.\n* Cover the pan and cook for 5-7 minutes, or until the karela is cooked through and the mixture is dry.\n* Serve hot with rice or roti.'

In [23]:
lines = response.split('\n')

gen_ingredients = []
gen_instructions = []
in_ingredients = False
in_instructions = False

for line in lines:
    line = line.strip() 
    if line == "Ingredients Used:":
        in_ingredients = True
        continue
    elif line == "Instructions:":
        in_ingredients = False
        in_instructions = True
        continue
    
    if in_ingredients and line.startswith("*"):
        gen_ingredients.append(line[1:].strip())
    elif in_instructions and line.startswith("*"):
        gen_instructions.append(line[1:].strip())

data['Processed_Ingredients'][0]

['salt',
 'amchur',
 'karela',
 'red chilli powder',
 'gram flour',
 'onion',
 'cumin seeds',
 'coriander powder',
 'turmeric powder',
 'sunflower oil']

In [26]:
actual_ingredients=data['Processed_Ingredients'][0]
gen_ingredients_lower = [ingredient.lower() for ingredient in gen_ingredients]

total_input_ingredients = len(actual_ingredients)
total_ingredients_used = len(gen_ingredients_lower)

relevant_ingredients_used = len(set(actual_ingredients).intersection(set(gen_ingredients_lower)))

precision = relevant_ingredients_used / total_ingredients_used
recall = relevant_ingredients_used / total_input_ingredients

print(precision)
print(recall)

1.0
0.9


In [40]:
data['TranslatedInstructions'][0].split('/n')

['To begin making the Masala Karela Recipe,de-seed the karela and slice.\nDo not remove the skin as the skin has all the nutrients.\nAdd the karela to the pressure cooker with 3 tablespoon of water, salt and turmeric powder and pressure cook for three whistles.\nRelease the pressure immediately and open the lids.\nKeep aside.Heat oil in a heavy bottomed pan or a kadhai.\nAdd cumin seeds and let it sizzle.Once the cumin seeds have sizzled, add onions and saute them till it turns golden brown in color.Add the karela, red chilli powder, amchur powder, coriander powder and besan.\nStir to combine the masalas into the karela.Drizzle a little extra oil on the top and mix again.\nCover the pan and simmer Masala Karela stirring occasionally until everything comes together well.\nTurn off the heat.Transfer Masala Karela into a serving bowl and serve.Serve Masala Karela along with Panchmel Dal and Phulka for a weekday meal with your family.\n']

In [46]:
gen_instructions_cleaned='\n '.join(gen_instructions)
gen_instructions_cleaned
from nltk.translate.bleu_score import sentence_bleu


# Tokenize the sentences
reference_tokens = data['TranslatedInstructions'][0].split('/n')  # Reference needs to be a list of token lists
candidate_tokens = gen_instructions
weights = (0.5, 0.5, 0, 0)
# Calculate BLEU score
bleu_score = sentence_bleu(reference_tokens, candidate_tokens,weights=weights)
bleu_score

0

In [44]:
candidate_tokens

['Heat sunflower oil in a pan over medium heat.',
 'Add cumin seeds and let them sizzle for a few seconds.',
 'Add finely chopped onion and sauté until it is translucent.',
 'Add karela (bitter gourd) and sauté for 2-3 minutes, or until it starts to soften.',
 'Add salt, red chilli powder, coriander powder, and turmeric powder, and mix well.',
 'Add gram flour (besan) and mix well.',
 'Add amchur (dried mango powder) and mix well.',
 'Cover the pan and cook for 5-7 minutes, or until the karela is cooked through and the mixture is dry.',
 'Serve hot with rice or roti.']

In [17]:
ingredients = re.findall(r'\* (.+)', response)

In [18]:
ingredients

['Salt',
 'Amchur',
 'Karela',
 'Red chilli powder',
 'Gram flour',
 'Onion',
 'Cumin seeds',
 'Coriander powder',
 'Turmeric powder',
 'Heat sunflower oil in a pan over medium heat.',
 'Add cumin seeds and let them sizzle for a few seconds.',
 'Add finely chopped onion and sauté until it is translucent.',
 'Add karela (bitter gourd) and sauté for 2-3 minutes, or until it starts to soften.',
 'Add salt, red chilli powder, coriander powder, and turmeric powder, and mix well.',
 'Add gram flour (besan) and mix well.',
 'Add amchur (dried mango powder) and mix well.',
 'Cover the pan and cook for 5-7 minutes, or until the karela is cooked through and the mixture is dry.',
 'Serve hot with rice or roti.']

In [5]:
data['Processed_Ingredients'][0]

['salt',
 'amchur',
 'karela',
 'red chilli powder',
 'gram flour',
 'onion',
 'cumin seeds',
 'coriander powder',
 'turmeric powder',
 'sunflower oil']

In [21]:
data['TranslatedInstructions'][0]

'To begin making the Masala Karela Recipe,de-seed the karela and slice.\nDo not remove the skin as the skin has all the nutrients.\nAdd the karela to the pressure cooker with 3 tablespoon of water, salt and turmeric powder and pressure cook for three whistles.\nRelease the pressure immediately and open the lids.\nKeep aside.Heat oil in a heavy bottomed pan or a kadhai.\nAdd cumin seeds and let it sizzle.Once the cumin seeds have sizzled, add onions and saute them till it turns golden brown in color.Add the karela, red chilli powder, amchur powder, coriander powder and besan.\nStir to combine the masalas into the karela.Drizzle a little extra oil on the top and mix again.\nCover the pan and simmer Masala Karela stirring occasionally until everything comes together well.\nTurn off the heat.Transfer Masala Karela into a serving bowl and serve.Serve Masala Karela along with Panchmel Dal and Phulka for a weekday meal with your family.\n'

In [3]:
data['Processed_Ingredients'][0]

['salt',
 'amchur',
 'karela',
 'red chilli powder',
 'gram flour',
 'onion',
 'cumin seeds',
 'coriander powder',
 'turmeric powder',
 'sunflower oil']

In [4]:
from langchain import LLMChain, PromptTemplate
from transformers import pipeline
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

access_token = ""

# Specify the model
model_name = "openai-community/gpt2"

# Try loading the fast tokenizer explicitly
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True, use_auth_token=access_token)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, use_auth_token=access_token)


# Define a custom prompt template for generating recipes
recipe_template = """
I have the following ingredients: {ingredients}.
I want to prepare a dish from {cuisine} cuisine.
Please generate a recipe using these ingredients with instructions.

Recipe:
"""

prompt = PromptTemplate(
    input_variables=["ingredients", "cuisine"],
    template=recipe_template
)

# LangChain LLM wrapper for Hugging Face model
class Llama2LLMChain(LLMChain):
    def __init__(self, model, tokenizer):
        super().__init__(llm=pipeline("text-generation", model=model, tokenizer=tokenizer))

    def _call(self, inputs):
        return super()._call(inputs)

# Initialize the LLaMA model as part of LangChain
llama_chain = Llama2LLMChain(model=model, tokenizer=tokenizer)


def generate_recipe(ingredients: list, cuisine: str) -> str:
    # Convert the ingredients list into a comma-separated string
    ingredients_str = ', '.join(ingredients)
    
    # Prepare input data for the LangChain
    input_data = {
        "ingredients": ingredients_str,
        "cuisine": cuisine
    }
    
    # Generate the recipe using the LLaMA model
    response = llama_chain(input_data)
    
    # Extract and return the generated text
    return response['text']

ingredients = data['Processed_Ingredients'][0]
cuisine = 'Indian'

recipe = generate_recipe(ingredients, cuisine)
print(recipe)



c:\Users\arnab\.conda\envs\data_science\Lib\site-packages\transformers\models\auto\tokenization_auto.py:786: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
c:\Users\arnab\.conda\envs\data_science\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
c:\Users\arnab\.conda\envs\data_science\Lib\site-packages\transformers\models\auto\auto_factory.py:469: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


ValidationError: 3 validation errors for Llama2LLMChain
prompt
  Field required [type=missing, input_value={'llm': <transformers.pip... at 0x000001E5AA1EA1E0>}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.7/v/missing
llm.is-instance[Runnable]
  Input should be an instance of Runnable [type=is_instance_of, input_value=<transformers.pipelines.t...t at 0x000001E5AA1EA1E0>, input_type=TextGenerationPipeline]
    For further information visit https://errors.pydantic.dev/2.7/v/is_instance_of
llm.is-instance[Runnable]
  Input should be an instance of Runnable [type=is_instance_of, input_value=<transformers.pipelines.t...t at 0x000001E5AA1EA1E0>, input_type=TextGenerationPipeline]
    For further information visit https://errors.pydantic.dev/2.7/v/is_instance_of